## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 15
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_15


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 25.1 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,14,2021,28.6,7.4,275.1,98.1,10.9,128.2,20.9,...,12.2,111.2,17.500000,13.500000,12.571429,1.357143,1.473837,0.119403,1.928571,2.454545
1,Tampa Bay Buccaneers,14,2021,29.3,7.2,304.9,100.4,10.7,97.6,21.9,...,9.7,89.0,15.571429,13.071429,13.071429,1.357143,1.869565,0.111111,2.571429,3.272727


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[14]

In [7]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
5,Los Angeles Rams,2021,week_15,0.281718,0.186919,0.238007,0.298126,0.251193
1,Tampa Bay Buccaneers,2021,week_15,0.218837,0.137272,0.145923,0.231287,0.183330
6,Buffalo Bills,2021,week_15,0.197093,0.097242,0.150851,0.227850,0.168259


In [10]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
5,Los Angeles Rams,2021,week_15,0.281718,0.186919,0.238007,0.298126,0.251193
1,Tampa Bay Buccaneers,2021,week_15,0.218837,0.137272,0.145923,0.231287,0.183330
6,Buffalo Bills,2021,week_15,0.197093,0.097242,0.150851,0.227850,0.168259
15,New England Patriots,2021,week_15,0.126465,0.126673,0.113115,0.198161,0.141104
11,San Francisco 49ers,2021,week_15,0.146355,0.070279,0.096187,0.228047,0.135217
0,Dallas Cowboys,2021,week_15,0.158663,0.089041,0.093884,0.178704,0.130073
8,Arizona Cardinals,2021,week_15,0.134858,0.072362,0.113365,0.193214,0.128450
14,Cincinnati Bengals,2021,week_15,0.122625,0.069523,0.120681,0.185578,0.124602
13,Green Bay Packers,2021,week_15,0.106356,0.062337,0.096560,0.203205,0.117115
2,Kansas City Chiefs,2021,week_15,0.131931,0.065370,0.070016,0.190352,0.114417
